# Init

In [1]:
#import aws
import os
import sys
import time
#import tensorflow as tf
import boto3

sys.path.insert(0, os.environ['HOME']+'/git0/cluster')
import util as u

import boto3
from collections import OrderedDict, defaultdict
ec2 = boto3.resource('ec2')


# G3 Instance peak performance

In [5]:
import os
import sys
import tensorflow as tf
import time

n = 8192
dtype = tf.float32
with tf.device("/gpu:0"):
    matrix1 = tf.Variable(tf.ones((n, n), dtype=dtype))
    matrix2 = tf.Variable(tf.ones((n, n), dtype=dtype))
    product = tf.matmul(matrix1, matrix2)


# avoid optimizing away redundant nodes
config = tf.ConfigProto(graph_options=tf.GraphOptions(optimizer_options=tf.OptimizerOptions(opt_level=tf.OptimizerOptions.L0)))
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
iters = 10

# pre-warming
sess.run(product.op)

start = time.time()
for i in range(iters):
  sess.run(product.op)
end = time.time()
ops = n**3 + (n-1)*n**2 # n^2*(n-1) additions, n^3 multiplications
elapsed = (end - start)
rate = iters*ops/elapsed/10**9
print('\n %d x %d matmul took: %.2f sec, %.2f G ops/sec' % (n, n,
                                                            elapsed/iters,
                                                            rate,))


 8192 x 8192 matmul took: 0.29 sec, 3792.15 G ops/sec


# Enable eager mode

In [2]:
from tensorflow.contrib.eager.python import tfe
tfe.enable_eager_execution()

ValueError: Do not call tfe.enable_eager_execution more than once in the same process. Note eager-mode methods such as tfe.run() also call tfe.enable_eager_execution.

# Instance control

http://boto3.readthedocs.io/en/latest/reference/services/ec2.html#instance

In [325]:
import sys, os
sys.path.insert(0, '/Users/yaroslav/git0/cluster')

In [37]:
import cluster_aws as toby_aws

In [3]:
instances = cluster_aws.LookupAwsInstances(instance_tag="whisper", ssh_key=os.environ['HOME']+'/d/yaroslav.pem')

In [4]:
instances

In [39]:
toby_aws.LookupAwsInstances("testjob")

[]

In [134]:
ec2 = boto3.resource('ec2')

instances = ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['terminated']}])

In [135]:
instances=list(instances)

In [137]:
instances

[ec2.Instance(id='i-08353b6df1352063d'),
 ec2.Instance(id='i-01fb7721c2de6de49'),
 ec2.Instance(id='i-0358efb3e93ea2c62'),
 ec2.Instance(id='i-01cef8d3f8c5abd08')]

In [35]:
ec2.instances.filter(InstanceIds=[]).all()

ec2.instancesCollection(ec2.ServiceResource(), ec2.Instance)

In [26]:
instance.name

AttributeError: 'ec2.Instance' object has no attribute 'name'

In [13]:
instance.tags

[{'Key': 'Name', 'Value': 'testjob'}]

In [133]:
[(i.id, tag['Value']) for i in instances for tag in i.tags if tag['Key'] == 'Name']

[('i-06a4a0d8e2236ae1c', 'testjob'),
 ('i-0b18d5c2bf06afa87', 'testjob'),
 ('i-0a279259ebf8e1c0b', 'testjob'),
 ('i-06977aebb6a40ce75', 'testjob')]

## Creating new instances with userdata

In [2]:
ec2.create_instances?

# Getting instance hostnames

In [148]:
instances=list(ec2.instances.filter(InstanceIds=['i-0be3150cdca115d44']))

In [151]:
instance.

AttributeError: 'ec2.Instance' object has no attribute 'hostname'

In [150]:
instance=instances[0]
instance.public_ip_address

'34.212.254.42'

In [156]:
instances=ec2.instances.filter(InstanceIds=['i-0be3150cdca115d44'])
for i in instances:
  print(i.id)

i-0be3150cdca115d44


In [159]:
for i in instances:
  print(i.id)

i-0be3150cdca115d44


In [152]:
import boto3
from collections import OrderedDict, defaultdict
ec2 = boto3.resource('ec2')


def get_private_instance_ip_map():
  """Return instance_id->private_ip map for all running instances."""
  
  ec2 = boto3.resource('ec2')

  # Get information for all running instances
  running_instances = ec2.instances.filter(Filters=[{
    'Name': 'instance-state-name',
    'Values': ['running']}])

  ec2info = OrderedDict()
  for instance in running_instances:
    name = ''
    for tag in instance.tags or []:
      if 'Name' in tag['Key']:
        name = tag['Value']
    print("%-15s %-20s  %s"%(instance.private_ip_address, instance.id, name))
    ec2info[instance.id] = instance.private_ip_address
    
  return ec2info

def get_public_instance_ip_map():
  """Return instance_id->private_ip map for all running instances."""
  
  ec2 = boto3.resource('ec2')

  # Get information for all running instances
  running_instances = ec2.instances.filter(Filters=[{
    'Name': 'instance-state-name',
    'Values': ['running']}])

  ec2info = OrderedDict()
  for instance in running_instances:
    name = ''
    for tag in instance.tags or []:
      if 'Name' in tag['Key']:
        name = tag['Value']
    print("%-15s %-20s  %s"%(instance.public_ip_address, instance.id, name))
    ec2info[instance.id] = instance.public_ip_address
    
  return ec2info

get_public_instance_ip_map()
#from pprint import pprint as pp
#pp(get_instance_ip_map())


34.209.255.229  i-047e3cc351ce00468   nodes.0726aa4c-pachydermcluster.kubernetes.com
52.36.167.177   i-0fd359996572dd7c6   testjob
52.40.109.190   i-0267a30cf58b2dc75   master-us-west-2a.masters.0726aa4c-pachydermcluster.kubernetes.com
52.89.53.239    i-0941a8cf4f1824c3f   nodes.90c444a2-pachydermcluster.kubernetes.com
52.37.131.117   i-06d8bd4450bf17b06   master-us-west-2a.masters.90c444a2-pachydermcluster.kubernetes.com
52.37.61.251    i-0b735ac755913f501   nodes.0726aa4c-pachydermcluster.kubernetes.com
52.27.162.71    i-07f875dcda4f055ed   testjob
34.212.254.42   i-0be3150cdca115d44   testjob
34.215.225.82   i-068c23912bcf71bfc   nodes.90c444a2-pachydermcluster.kubernetes.com


OrderedDict([('i-047e3cc351ce00468', '34.209.255.229'),
             ('i-0fd359996572dd7c6', '52.36.167.177'),
             ('i-0267a30cf58b2dc75', '52.40.109.190'),
             ('i-0941a8cf4f1824c3f', '52.89.53.239'),
             ('i-06d8bd4450bf17b06', '52.37.131.117'),
             ('i-0b735ac755913f501', '52.37.61.251'),
             ('i-07f875dcda4f055ed', '52.27.162.71'),
             ('i-0be3150cdca115d44', '34.212.254.42'),
             ('i-068c23912bcf71bfc', '34.215.225.82')])

# Sending files

In [125]:
import paramiko
def SshToHost(hostname,
              ssh_key=None,
              username='ubuntu',
            retry=10):

  """Create ssh connection to host

  Creates and returns and ssh connection to the host passed in.  

  Args:
    hostname: host name or ip address of the system to connect to.
    retry: number of time to retry.
    ssh_key: full path to the ssk hey to use to connect.
    username: username to connect with.

  returns SSH client connected to host.

  """

  k = paramiko.RSAKey.from_private_key_file(ssh_key)
  
  ssh_client = paramiko.SSHClient()
  ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

  counter = retry
  while counter > 0:
    try:
      ssh_client.connect(hostname=hostname, username=username, pkey=k)
      break
    except Exception as e:
      counter = counter - 1
      print('Exception connecting to host via ssh (could be a timeout):'.format(e))
      if counter == 0:
        print('Got impatient with retrying ssh to host. Time to give up.')
        return None

  return ssh_client

In [126]:
username='ubuntu'
KEY_PATH = os.environ['HOME']+'/d/yaroslav.pem'
ssh_key=KEY_PATH
hostname = "52.27.162.71"
ssh_client = SshToHost(hostname, ssh_key)
sftp = ssh_client.open_sftp()
sftp.put('/temp/hi.txt', 'hi.txt')
stdin,stdout,stderr=ssh_client.exec_command('cat hi.txt')
print(stdout.read().decode("ascii"))

In [154]:
os.path.basename(KEY_PATH)

'yaroslav.pem'

hi
testing



In [165]:
time.time()

1510529639.275616

In [180]:
import struct
value = time.time()
ba = bytearray(struct.pack("d", value))  
import base64
b16=base64.b16encode(ba)

In [181]:
value

1510530098.287752

In [182]:
b16

b'876A920C3782D641'

In [183]:
ba

bytearray(b'\x87j\x92\x0c7\x82\xd6A')

In [186]:
ba

bytearray(b'\x87j\x92\x0c7\x82\xd6A')

In [189]:
b16s = b16.decode('ascii')

In [187]:
struct.unpack('d', base64.b16decode(b16))

(1510530098.287752,)

In [190]:
struct.unpack('d', base64.b16decode(b16s))

(1510530098.287752,)

In [192]:
'asdfasdf-asdf-asdf'.rsplit('-',1)

['asdfasdf-asdf', 'asdf']

In [127]:
ssh_client.exec_command('ls')

(<paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0xf4a5978 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0xf4a5978 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0xf4a5978 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>)

In [99]:
ssh_client.exec_command('cd efs', get_pty=True)

(<paramiko.ChannelFile from <paramiko.Channel 16 (open) window=2097152 -> <paramiko.Transport at 0xf50e9b0 (cipher aes128-ctr, 128 bits) (active; 2 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 16 (open) window=2097152 -> <paramiko.Transport at 0xf50e9b0 (cipher aes128-ctr, 128 bits) (active; 2 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 16 (open) window=2097152 -> <paramiko.Transport at 0xf50e9b0 (cipher aes128-ctr, 128 bits) (active; 2 open channel(s))>>>)

In [121]:
# from https://stackoverflow.com/questions/35821184/implement-an-interactive-shell-over-ssh-in-python-using-paramiko
import paramiko
import re


class Shell:
    def __init__(self, host, ssh_key, username='ubuntu', retry=10):
        k = paramiko.RSAKey.from_private_key_file(ssh_key)

        self.ssh = paramiko.SSHClient()
        self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        counter = retry
        while counter > 0:
          try:
            self.ssh.connect(hostname=hostname, username=username, pkey=k)
            break
          except Exception as e:
            counter = counter - 1
            print('Exception connecting to host via ssh (could be a timeout):'.format(e))
        if counter == 0:
          print('Got impatient with retrying ssh to host. Time to give up.')
          return None

        shell = self.ssh.invoke_shell()


    def __del__(self):
        self.ssh.close()

    def cmd(self, cmd):
        """

        :param cmd: the command to be executed on the remote computer
        :examples:  execute('ls')
                    execute('finger')
                    execute('cd folder_name')
        """

        cmd = cmd.strip('\n')
        shell.send(cmd+'\n')
        
        self.stdin.write(cmd + '\n')
        self.stdin.flush()

In [122]:
shell = Shell(hostname, ssh_key)

In [123]:
shell.cmd('ls')

In [124]:
ss=shell.stdout.read()

KeyboardInterrupt: 

In [116]:
shell.cmd('ls')
print(shell.stdout.read().decode("ascii"))

KeyboardInterrupt: 

In [ ]:
shell.stdout

In [113]:
stdout

['anaconda3                                hi.txt\n',
 'Anaconda3-4.4.0-Linux-x86_64.sh          home\n',
 'cuda-repo-ubuntu1604_8.0.61-1_amd64.deb  libcudnn5_5.1.10-1_cuda8.0_amd64.deb\n',
 'd                                        libcudnn6_6.0.21-1_cuda8.0_amd64.deb\n',
 'Dropbox                                  pytorch_example\n',
 'early-eager                              pytorch_example.tar\n',
 'efs                                      #test.txt#\n',
 'efs2                                     whisper\n',
 'git0\n',
 'ubuntu@ip-172-31-37-145:~$ echo end of stdOUT buffer. finished with exit status  $?\n']

In [ ]:
channel = ssh_client.invoke_shell()
stdin = channel.makefile('wb')
stdout = channel.makefile('r')

In [96]:
stdin,stdout,stderr=ssh_client.exec_command('cd efs', get_pty=True)
print(stdout.read().decode("ascii"))

In [97]:
stdin,stdout,stderr=ssh_client.exec_command('ls', get_pty=True)
print(stdout.read().decode("ascii"))

anaconda3				 hi.txt
Anaconda3-4.4.0-Linux-x86_64.sh		 home
cuda-repo-ubuntu1604_8.0.61-1_amd64.deb  libcudnn5_5.1.10-1_cuda8.0_amd64.deb
d					 libcudnn6_6.0.21-1_cuda8.0_amd64.deb
Dropbox					 pytorch_example
early-eager				 pytorch_example.tar
efs					 #test.txt#
efs2					 whisper
git0



In [92]:
sftp = ssh_client.open_sftp()
sftp.put('/temp/hi.txt', 'hi.txt')

<SFTPAttributes: [ size=11 uid=1000 gid=1000 mode=0o100664 atime=1510523151 mtime=1510523217 ]>

# Placement groups

In [129]:
res = client.create_placement_group(GroupName="test3", Strategy='cluster')

In [130]:
res

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 19:49:22 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '4f745b13-6567-4272-8197-fe4a6acf5ef7',
  'RetryAttempts': 0}}

In [153]:

def _create_ec2_client():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.client('ec2', region_name=REGION)

def get_placement_group_dict():
  """Returns dictionary of {placement_group_name: state}"""

  client = _create_ec2_client()
  response = client.describe_placement_groups()
  assert _is_good_response(response)

  result = OrderedDict()
  for placement_group_response in response['PlacementGroups']:
    key = placement_group_response['GroupName']
    assert key not in result, ("Duplicate placement group " +
                               placement_group_name)
    result[key] = placement_group_response['State']
  return response

In [154]:
get_placement_group_dict()

{'PlacementGroups': [{'GroupName': 'inet3',
   'State': 'available',
   'Strategy': 'cluster'},
  {'GroupName': 'amazon1', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'default', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'test3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy_head', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded', 'State'

In [ ]:
delete_place

In [127]:
client = create_ec2_client()


In [1]:
import boto3
ec2 = boto3.client('ec2')

In [9]:
result=ec2.describe_placement_groups(GroupNames=["beefy2"])

In [13]:
result['PlacementGroups'][0]['State']=='available'

True

In [9]:
result['PlacementGroups'][0]

{'GroupName': 'gpu12', 'State': 'available', 'Strategy': 'cluster'}

In [2]:
ec2.describe_placement_groups(GroupNames=['gpu3'])

{'PlacementGroups': [{'GroupName': 'gpu3',
   'State': 'available',
   'Strategy': 'cluster'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 28 Nov 2017 18:51:03 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'a92ecfb4-7897-4def-8edc-6bdccfe9ac61',
  'RetryAttempts': 0}}

In [10]:
ec2.delete_placement_group(GroupName='test')

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Fri, 17 Nov 2017 05:26:01 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '9f5590d2-2b43-473b-bf03-0045eff22408',
  'RetryAttempts': 0}}

In [17]:
try:
  res = client.create_placement_group(GroupName='gpu3', Strategy='cluster')
except Exception as e:
  aaa = e

In [160]:
client.delete_placement_group(GroupName="beefy")

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 13 Dec 2017 20:27:50 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '4d501789-b047-47e8-9354-4513c345e722',
  'RetryAttempts': 0}}

In [20]:
aaa.response['Error']['Code']

'InvalidPlacementGroup.Duplicate'

In [21]:
str(aaa)

"An error occurred (InvalidPlacementGroup.Duplicate) when calling the CreatePlacementGroup operation: The placement group 'gpu3' already exists."

In [ ]:
def MaybeCreatePlacementGroup(name='tf_bm'):
  client = boto3.client('ec2')
  try:
    client.describe_placement_groups(GroupNames=[name])
  except boto3.exceptions.botocore.exceptions.ClientError as e:
    res = client.create_placement_group(GroupName=name, Strategy='cluster')

  counter = 0
  while True:
    try:
      res = client.describe_placement_groups(GroupNames=[name])
      if res['PlacementGroups'][0]['State'] == 'available':
        break
    except:
      pass
    counter = counter + 1
    if counter >= 10:
      print('Failed to create placement group %s' % name)
    time.sleep(10)


def DeletePlacementGroup(name='tf_bm'):
  client = boto3.client('ec2')
  try:
    client.describe_placement_groups(GroupNames=[name])
  except boto3.exceptions.botocore.exceptions.ClientError as e:
    print("Placement group %s doesn't exit." % name)
    return

  # Not sure whether delete_placement_group would throw or not.
  res = client.delete_placement_group(GroupName=name)
  if res['ResponseMetadata']['HTTPStatusCode'] != 200:
    print('Failed to delete placement group %s' % name)

# Instance limits using limit checker

In [9]:
ec2 = boto3.client('ec2')
response = ec2.describe_instances()

instance_list = []
for reservation in response['Reservations']:
  for instance in reservation['Instances']:
    instance_list.append(instance)

In [12]:
inst = 0
for i in instance_list:
  print(i['InstanceType'], i['State']['Name'])
  if i['InstanceType'].startswith('p2'):
    inst = i
    break

r4.xlarge running
c5.2xlarge terminated
c5.2xlarge terminated
c5.2xlarge terminated
c5.2xlarge terminated
p2.8xlarge running


In [10]:
from awslimitchecker.checker import AwsLimitChecker
c = AwsLimitChecker()

awslimitchecker 2.0.0 is AGPL-licensed free software; all users have a right to the full source code of this version. See <https://github.com/jantman/awslimitchecker>


In [11]:
c.find_usage()

ERROR - unknown instance type 'c5.2xlarge'; not counting
ERROR - unknown instance type 'c5.2xlarge'; not counting
ERROR - unknown instance type 'c5.2xlarge'; not counting
ERROR - unknown instance type 'c5.2xlarge'; not counting


In [12]:
limits=c.get_limits()

In [18]:
ll=limits['EC2']['Running On-Demand c5.xlarge instances']

KeyError: 'Running On-Demand c5.xlarge instances'

In [14]:
usage = ll.get_current_usage()[0]

In [15]:
usage.get_value()

0

In [16]:
machine_limit = limits['EC2']['Running On-Demand r4.xlarge instances']

In [17]:
machine_limit.get_limit_source()

0

In [50]:
temp=machine_limit.get_current_usage()[0]

In [53]:
temp.aws_type

'AWS::EC2::Instance'

In [47]:
machine_limit.get_current_usage_str()

'6'

In [44]:
machine_limit.get_limit()

20

# Instance limits using boto3 

In [27]:
ec2.describe_account_attributes(AttributeNames=["max-instances"])

{'AccountAttributes': [{'AttributeName': 'max-instances',
   'AttributeValues': [{'AttributeValue': '20'}]}],
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 28 Nov 2017 21:11:48 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '16d4bdaa-fd5e-4376-953d-a13f93944fc8',
  'RetryAttempts': 0}}

# Instance state query

In [24]:
  ec2 = boto3.client('ec2')
  response = ec2.describe_instances()


In [16]:
  instance_list = []
  for reservation in response['Reservations']:
    for instance in reservation['Instances']:
      instance_list.append(((instance['LaunchTime']), instance))


In [19]:
instance_list[0][1]

{'AmiLaunchIndex': 1,
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
   'Ebs': {'AttachTime': datetime.datetime(2017, 12, 2, 2, 3, 33, tzinfo=tzutc()),
    'DeleteOnTermination': True,
    'Status': 'attached',
    'VolumeId': 'vol-0736954def22c0445'}}],
 'ClientToken': '',
 'EbsOptimized': False,
 'EnaSupport': True,
 'Hypervisor': 'xen',
 'ImageId': 'ami-bf4193c7',
 'InstanceId': 'i-0af3d8d828b9c4cb9',
 'InstanceType': 'c5.18xlarge',
 'KeyName': 'yaroslav',
 'LaunchTime': datetime.datetime(2017, 12, 2, 2, 3, 32, tzinfo=tzutc()),
 'Monitoring': {'State': 'disabled'},
 'NetworkInterfaces': [{'Association': {'IpOwnerId': 'amazon',
    'PublicDnsName': 'ec2-35-167-132-34.us-west-2.compute.amazonaws.com',
    'PublicIp': '35.167.132.34'},
   'Attachment': {'AttachTime': datetime.datetime(2017, 12, 2, 2, 3, 32, tzinfo=tzutc()),
    'AttachmentId': 'eni-attach-a4470155',
    'DeleteOnTermination': True,
    'DeviceIndex': 0,
    'Status': 'attached'},
   'De

In [20]:
instance_list[0][1]["State"]["Name"]

'running'

# Find out size of instance volume

In [23]:
ec2 = u.create_ec2_resource()

In [24]:
instances=list(ec2.instances.all())

In [26]:
inst=instances[0]

In [27]:
inst.launch_time

datetime.datetime(2018, 1, 15, 20, 16, 6, tzinfo=tzutc())

In [29]:
vol=ec2.Volume('vol-0a98ee5f7f155b2b7')

In [32]:
client=u.create_ec2_client()

In [33]:
client.modify_volume?

In [23]:
import boto3
ec2 = boto3.resource('ec2', region_name='us-east-1')
instance_id='i-057cc0bd9fb49d07e'
instance =  ec2.Instance(instance_id)
volumes = instance.volumes.all()
for v in volumes:
    print("Instance id: %s, disk size: %.1f GB, state: %s"%(v.id, v.size, v.state))


Instance id: vol-01564523457cf1ffb, disk size: 500.0 GB, state: in-use


In [39]:
def get_name(instance_response):
  names = [entry['Value'] for entry in instance_response.get('Tags',[]) if
           entry['Key']=='Name']
  if not names:
    names = ['']
  assert len(names)==1
  return names[0]

from collections import defaultdict
ec2 = boto3.client('ec2')
response = ec2.describe_instances()
instances = defaultdict(lambda: [])
for reservation in response['Reservations']:
  for instance_response in reservation['Instances']:
    instances[get_name(instance_response)].append(instance_response)


In [41]:
instances['p3']

[{'AmiLaunchIndex': 0,
  'Architecture': 'x86_64',
  'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
    'Ebs': {'AttachTime': datetime.datetime(2017, 12, 4, 20, 33, 1, tzinfo=tzutc()),
     'DeleteOnTermination': True,
     'Status': 'attached',
     'VolumeId': 'vol-090c4a23e59922b4b'}}],
  'ClientToken': '',
  'EbsOptimized': False,
  'EnaSupport': True,
  'Hypervisor': 'xen',
  'ImageId': 'ami-f1e73689',
  'InstanceId': 'i-0565a2e6b11533428',
  'InstanceType': 'p3.16xlarge',
  'KeyName': 'yaroslav',
  'LaunchTime': datetime.datetime(2017, 12, 4, 20, 33, tzinfo=tzutc()),
  'Monitoring': {'State': 'disabled'},
  'NetworkInterfaces': [{'Association': {'IpOwnerId': 'amazon',
     'PublicDnsName': 'ec2-52-26-171-202.us-west-2.compute.amazonaws.com',
     'PublicIp': '52.26.171.202'},
    'Attachment': {'AttachTime': datetime.datetime(2017, 12, 4, 20, 33, tzinfo=tzutc()),
     'AttachmentId': 'eni-attach-d0085836',
     'DeleteOnTermination': True,
     'DeviceIndex': 0,
     'Status

# Find AMI

In [20]:
import boto3
ec2 = boto3.resource('ec2', region_name='us-west-2')

In [22]:
for image in list(ec2.images.filter(Filters=[{'Name':'name', 'Values':['ubuntu-14.04*']}]).all()):
  print(image.name)
  break

ubuntu-14.04-VC4.1.3-1-62ab724f-833e-4666-99d0-21e3ec7ad743-ami-d569eac2.3


In [26]:
image.name

'ubuntu-14.04-VC4.1.3-1-62ab724f-833e-4666-99d0-21e3ec7ad743-ami-d569eac2.3'

In [ ]:
https://aws.amazon.com/marketplace/pp/B077GCH38C?qid=1512418106283&sr=0-2&ref_=srh_res_product_title

In [27]:
image.id

'ami-03cf0263'

In [32]:
image = list(ec2.images.filter(Filters=[{"Name":"Id", "Values": ["ami-405ade3a"]}]).all())[0]

ClientError: An error occurred (InvalidParameterValue) when calling the DescribeImages operation: The filter 'Id' is invalid

# Debugging paramiko

In [47]:
import paramiko
username='ubuntu'
ssh_key="/Users/yaroslav/d/yaroslav.pem"
hostname='ec2-52-34-58-144.us-west-2.compute.amazonaws.com'

print("ssh_to_host %s@%s"%(username, hostname))
k = paramiko.RSAKey.from_private_key_file(ssh_key)
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname=hostname, username=username, pkey=k)


ssh_to_host ubuntu@ec2-52-34-58-144.us-west-2.compute.amazonaws.com


In [49]:
for i in range(100):
  ssh_client.exec_command("tmux send-keys -t tmux 'sleep 3; echo %d' Enter"%(i,))

In [46]:
ssh_client.exec_command("tmux send-keys -t tmux 'ls' Enter")

(<paramiko.ChannelFile from <paramiko.Channel 11 (open) window=2097152 -> <paramiko.Transport at 0xb2e0f60 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 11 (open) window=2097152 -> <paramiko.Transport at 0xb2e0f60 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 11 (open) window=2097152 -> <paramiko.Transport at 0xb2e0f60 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>)

# New region setup

In [47]:
import boto3
import os
def _create_ec2_client():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.client('ec2', region_name=REGION)

def _create_efs_client():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.client('efs', region_name=REGION)

def _create_efs_resource():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.resource('efs', region_name=REGION)

def _create_ec2_resource():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.resource('ec2',region_name=REGION)

def _is_good_response(response):
  """Helper method to check if boto3 call was a success."""
  
  return response["ResponseMetadata"]['HTTPStatusCode'] == 200

def _get_name(tags):
  """Helper utility to extract name out of tags dictionary.
      [{'Key': 'Name', 'Value': 'nexus'}] -> 'nexus'
 
     Assert fails if there's more than one name.
     Returns '' if there's less than one name.
  """
  
  names = [entry['Value'] for entry in tags if entry['Key']=='Name']
  if not names:
    return ''
  if len(names)>1:
    assert False, "have more than one name: "+str(names)
  return names[0]


## Creating new security group

In [241]:
import boto3
REGION='us-west-1'
ec2 = boto3.resource('ec2',region_name=REGION)

In [243]:
security_groups=list(vpc.security_groups.all())

In [245]:
security_groups

[]

In [422]:
  DEFAULT_NAME = 'nexus'
  security_group = u.get_security_group_dict()[DEFAULT_NAME]
  keypair = u.get_keypair_dict()[DEFAULT_NAME]


In [423]:
keypair.name

'nexus'

In [420]:
security_group = u.get_security_group_dict()['nexus']

In [421]:
security_group.group_name

'nexus'

In [411]:
subnets = list(vpc.subnets.all())

In [416]:
ec2.create_instances?

In [415]:
client.describe_vpcs()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 19 Dec 2017 23:44:30 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'c5de6a58-3c5f-48ba-a2a6-eed01188d239',
  'RetryAttempts': 0},
 'Vpcs': [{'CidrBlock': '172.31.0.0/16',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-984c4ff1',
     'CidrBlock': '172.31.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'DhcpOptionsId': 'dopt-4689da22',
   'InstanceTenancy': 'default',
   'IsDefault': True,
   'State': 'available',
   'VpcId': 'vpc-79c4f61d'}]}

In [259]:
security_groups[0].group_name

'nexus'

In [260]:
security_group = security_groups[0]

In [261]:
security_group.tags

[{'Key': 'Name', 'Value': 'nexus'}]

In [ ]:
c.get_name(security_groups[0].)

## Creating network

In [50]:
vpc = ec2.create_vpc(CidrBlock='192.168.0.0/16')

In [51]:
vpc.create_tags(Tags=[{"Key": "Name", "Value": "nexus_vpc"}])

[ec2.Tag(resource_id='vpc-2bb1584c', key='Name', value='nexus_vpc')]

In [53]:
client.describe_vpcs()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 19 Dec 2017 00:00:50 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '29be9d8e-8ae1-45f0-80d4-53d6a1aae72d',
  'RetryAttempts': 0},
 'Vpcs': [{'CidrBlock': '192.168.0.0/16',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-90cb8dfa',
     'CidrBlock': '192.168.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'DhcpOptionsId': 'dopt-fcb80385',
   'InstanceTenancy': 'default',
   'IsDefault': False,
   'State': 'available',
   'Tags': [{'Key': 'Name', 'Value': 'nexus'}],
   'VpcId': 'vpc-1a054562'},
  {'CidrBlock': '172.31.0.0/16',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-4805d222',
     'CidrBlock': '172.31.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'DhcpOptionsId': 'dopt-fcb80385',
   'InstanceTenancy': 'default',
   'IsDefault': Tr

In [54]:
vpc.wait_until_available()

In [55]:
    ig = ec2.create_internet_gateway()

In [56]:
    ig.attach_to_vpc(VpcId=vpc.id)


{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 19 Dec 2017 00:01:21 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'c860ca96-78d0-420b-8ec0-a1279fc6bf5e',
  'RetryAttempts': 0}}

In [117]:
os.environ['AWS_DEFAULT_REGION']='us-west-1'

In [118]:
  from collections import OrderedDict
  client = _create_ec2_client()
  response = client.describe_vpcs()
  assert _is_good_response(response)

  result = OrderedDict()
  ec2 = _create_ec2_resource()
  for vpc_response in response['Vpcs']:
    key = _get_name(vpc_response.get('Tags', []))
    if not key:  # skip VPC's that don't have a name assigned
      continue
    
    result[key] = ec2.Vpc(vpc_response['VpcId'])

  print(result)


OrderedDict([('nexus_vpc', ec2.Vpc(id='vpc-14876473'))])


In [119]:
client = _create_ec2_client()

In [124]:
vpc.id

'vpc-2bb1584c'

In [130]:
vpc.get_available_subresources()

[]

In [132]:
vpc.delete()

ClientError: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-2bb1584c' has dependencies and cannot be deleted.

In [139]:
ec2.subnets.all()[0]

TypeError: 'ec2.subnetsCollection' object does not support indexing

In [140]:
response = client.delete_vpc(VpcId=vpc.id)

ClientError: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-2bb1584c' has dependencies and cannot be deleted.

In [152]:
list(vpc.route_tables.all())

[ec2.RouteTable(id='rtb-f4ba3693'), ec2.RouteTable(id='rtb-a8b438cf')]

In [150]:
for r in vpc.route_tables.all():
  print(r)

ec2.RouteTable(id='rtb-f4ba3693')
ec2.RouteTable(id='rtb-a8b438cf')


In [158]:
subnets = list(vpc.subnets.all())

In [159]:
subnets

[]

In [157]:
subnet = subnets[1]

IndexError: list index out of range

In [156]:
subnet.delete()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 19 Dec 2017 01:43:58 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'a322b866-3966-495f-9b6f-b05ceca514a5',
  'RetryAttempts': 0}}

In [72]:
kk.issubset(ig.attachments[0].keys())

True

In [73]:
def _get1(items, **kwargs):
  """Helper method to extract values, ie
  response = [{'State': 'available', 'VpcId': 'vpc-2bb1584c'}]
  _get1(response, State=-1, VpcId='vpc-2bb1584c') #=> 'available'"""

  # find the value of attribute to return
  query_arg = None
  for arg, value in kwargs.items():
    if value == -1:
      assert query_arg is None, "Only single query arg (-1 valued) is allowed"
      query_arg = arg
  result = []
  
  filterset = set(kwargs.keys())
  for item in items:
    match = True
    assert filterset.issubset(item.keys()), "Filter set contained %s which was not in record %s" %(filterset.difference(item.keys()),
                                                                                                  item)
    for arg in item:
      if arg == query_arg:
        continue
      if arg in kwargs:
        if item[arg] != kwargs[arg]:
          match = False
          break
    if match:
      result.append(item[query_arg])
  assert len(result) <= 1, "%d values matched %s, only allow 1" % (len(result), kwargs)
  if result:
    return result[0]
  return None

In [74]:
_get1(ig.attachments, State=-1, VpcId='vpc-2bb1584c')

'available'

In [75]:
ig.attachments?

In [13]:
vpc-14876473

TypeError: unsupported operand type(s) for -: 'ec2.Vpc' and 'int'

In [76]:
client = boto3.client('ec2')
route_table = vpc.create_route_table()

In [77]:
vpc.create_route_table?

In [79]:
route_table.load()

In [84]:
def _make_name(name):
  return [{'Key': 'Name', 'Value': name}]

In [87]:
route_table.create_tags(Tags=_make_name("nexus"))

[ec2.Tag(resource_id='rtb-a8b438cf', key='Name', value='nexus')]

In [86]:
route_table.create_tags?

In [88]:
route_table.tags

[{'Key': 'Name', 'Value': 'nexus'}]

In [89]:
route = route_table.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=ig.id
)


In [91]:
route.load()

AttributeError: 'ec2.Route' object has no attribute 'load'

In [94]:
route_table.get_available_subresources()

[]

In [95]:
route_table.routes

[ec2.Route(route_table_id='rtb-a8b438cf', destination_cidr_block='192.168.0.0/16'),
 ec2.Route(route_table_id='rtb-a8b438cf', destination_cidr_block='0.0.0.0/0')]

In [101]:
for i in range(5):
  if i == 6:
    break
else:
  print("Failed")

Failed


In [160]:
    subnet = vpc.create_subnet(CidrBlock='192.168.1.0/20')

In [161]:
subnet.id

'subnet-76901911'

In [163]:
route_tables = list(vpc.route_tables.all())

In [164]:
route_tables

[ec2.RouteTable(id='rtb-f4ba3693'), ec2.RouteTable(id='rtb-a8b438cf')]

In [165]:
route_table = route_tables[0]

In [167]:
route_table.id

'rtb-f4ba3693'

In [209]:
security_groups = list(vpc.security_groups.all())

In [212]:
security_groups[1].delete()

IndexError: list index out of range

In [214]:
security_groups[0]

ec2.SecurityGroup(id='sg-4e8c6c37')

In [ ]:
subnet.

In [215]:
security_groups[0].delete()

ClientError: An error occurred (CannotDelete) when calling the DeleteSecurityGroup operation: the specified group: "sg-4e8c6c37" name: "default" cannot be deleted by a user

In [170]:
route_table.routes

[ec2.Route(route_table_id='rtb-f4ba3693', destination_cidr_block='192.168.0.0/16')]

In [172]:
route_table.routes[0].delete()

ClientError: An error occurred (InvalidParameterValue) when calling the DeleteRoute operation: cannot remove local route 192.168.0.0/16 in route table rtb-f4ba3693

In [175]:
gateways = list(vpc.internet_gateways.all())

In [176]:
gateway = gateways[0]

In [180]:
gateway.detach_from_vpc(VpcId=vpc.id)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 19 Dec 2017 01:51:15 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '230d20fa-3cf3-4344-afe6-159d0ac6f522',
  'RetryAttempts': 0}}

In [186]:
list(vpc.internet_gateways.all())

[]

In [193]:
route_tables = list(vpc.route_tables.all())

In [195]:
route_table = route_tables[0]

In [199]:
route = route_table.routes[0]

In [200]:
route

ec2.Route(route_table_id='rtb-f4ba3693', destination_cidr_block='192.168.0.0/16')

In [201]:
route.delete()

ClientError: An error occurred (InvalidParameterValue) when calling the DeleteRoute operation: cannot remove local route 192.168.0.0/16 in route table rtb-f4ba3693

In [206]:
list(vpc.security_groups.all())

[ec2.SecurityGroup(id='sg-4e8c6c37')]

In [104]:
vpc.create_subnet?

In [97]:
rr = route_table.routes[1]

In [99]:
rr.destination_cidr_block

'0.0.0.0/0'

In [ ]:
ec.RouteTable('rtb-3b530d5d').routes

In [92]:
route.state

ResourceLoadException: ec2.Route has no load method

In [58]:
response = client.describe_vpcs()

In [63]:
response["ResponseMetadata"]['HTTPStatusCode']

200

In [64]:
response['Vpcs']

[{'CidrBlock': '172.31.0.0/16',
  'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-984c4ff1',
    'CidrBlock': '172.31.0.0/16',
    'CidrBlockState': {'State': 'associated'}}],
  'DhcpOptionsId': 'dopt-4689da22',
  'InstanceTenancy': 'default',
  'IsDefault': True,
  'State': 'available',
  'VpcId': 'vpc-79c4f61d'},
 {'CidrBlock': '192.168.0.0/16',
  'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-35a2505d',
    'CidrBlock': '192.168.0.0/16',
    'CidrBlockState': {'State': 'associated'}}],
  'DhcpOptionsId': 'dopt-4689da22',
  'InstanceTenancy': 'default',
  'IsDefault': False,
  'State': 'available',
  'Tags': [{'Key': 'Name', 'Value': 'nexus_vpc'}],
  'VpcId': 'vpc-14876473'}]

In [80]:
os.environ['AWS_DEFAULT_REGION']='us-west-1'

In [77]:
def get_name(tags):
  """[{'Key': 'Name', 'Value': 'nexus'}] -> 'nexus'"""
  names = [entry['Value'] for entry in tags if entry['Key']=='Name']
  if not names:
    names = ['']
  assert len(names)==1, "have more than one name: "+str(names)
  return names[0]

def create_ec2_client():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.client('ec2', region_name=REGION)

def create_ec2_resource():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.resource('ec2',region_name=REGION)

def get_vpc_dict():
  """Returns dictionary of {name:[vpc1,vpc2, ],...} """
  from collections import defaultdict
  result = defaultdict(list)
  client = create_ec2_client()
  response = client.describe_vpcs()
  assert response["ResponseMetadata"]['HTTPStatusCode'] == 200
  for vpc_response in response['Vpcs']:
    result[get_name(vpc_response.get('Tags', []))].append(vpc_response)
  return result

In [85]:
vpc.id

'vpc-14876473'

In [82]:
get_vpc_dict()

defaultdict(list,
            {'': [{'CidrBlock': '172.31.0.0/16',
               'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-984c4ff1',
                 'CidrBlock': '172.31.0.0/16',
                 'CidrBlockState': {'State': 'associated'}}],
               'DhcpOptionsId': 'dopt-4689da22',
               'InstanceTenancy': 'default',
               'IsDefault': True,
               'State': 'available',
               'VpcId': 'vpc-79c4f61d'}],
             'nexus_vpc': [{'CidrBlock': '192.168.0.0/16',
               'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-35a2505d',
                 'CidrBlock': '192.168.0.0/16',
                 'CidrBlockState': {'State': 'associated'}}],
               'DhcpOptionsId': 'dopt-4689da22',
               'InstanceTenancy': 'default',
               'IsDefault': False,
               'State': 'available',
               'Tags': [{'Key': 'Name', 'Value': 'nexus_vpc'}],
               'VpcId': 'vpc-14876473'}]})

In [84]:
'nexus' in get_vpc_dict()

False

In [79]:
client = create_ec2_client()
response = client.describe_vpcs()
response  

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 05:20:58 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '1fe9496b-f3af-4c48-9841-35c9b0172bf6',
  'RetryAttempts': 0},
 'Vpcs': [{'CidrBlock': '172.31.0.0/16',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-4805d222',
     'CidrBlock': '172.31.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'DhcpOptionsId': 'dopt-fcb80385',
   'InstanceTenancy': 'default',
   'IsDefault': True,
   'State': 'available',
   'VpcId': 'vpc-2b72a053'}]}

In [55]:
client.describe_vpcs(VpcIds=[vpc.id])

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 05:06:27 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'aaf8c91c-8ef1-425b-87e9-e8c13a17ecf5',
  'RetryAttempts': 0},
 'Vpcs': [{'CidrBlock': '192.168.0.0/16',
   'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-35a2505d',
     'CidrBlock': '192.168.0.0/16',
     'CidrBlockState': {'State': 'associated'}}],
   'DhcpOptionsId': 'dopt-4689da22',
   'InstanceTenancy': 'default',
   'IsDefault': False,
   'State': 'available',
   'Tags': [{'Key': 'Name', 'Value': 'nexus_vpc'}],
   'VpcId': 'vpc-14876473'}]}

In [187]:
def get_vpc_dict():
  """Returns dictionary of named VPCs {name: vpc_id}

  Assert fails if there's more than one VPC with same name."""

  client = _create_ec2_client()
  response = client.describe_vpcs()
  assert _is_good_response(response)

  result = OrderedDict()
  for vpc_response in response['Vpcs']:
    key = _get_name(vpc_response.get('Tags', []))
    if not key:  # skip VPC's that don't have a name assigned
      continue
    
    assert key not in result, ("Duplicate VPC group " + key)
    result[key] = vpc_response['VpcId']
    
  return result


In [189]:
def _get_name(tags):
  """Helper utility to extract name out of tags dictionary.
      [{'Key': 'Name', 'Value': 'nexus'}] -> 'nexus'
 
     Assert fails if there's more than one name.
     Returns '' if there's less than one name.
  """
  
  names = [entry['Value'] for entry in tags if entry['Key']=='Name']
  if not names:
    names = ['']
  assert len(names)==1, "have more than one name: "+str(names)
  return names[0]

get_vpc_dict()

OrderedDict([('282e747a-pachydermcluster.kubernetes.com', 'vpc-f683678f'),
             ('yong', 'vpc-d91a23bf'),
             ('0726aa4c-pachydermcluster.kubernetes.com', 'vpc-14500972')])

In [191]:
vvv=ec2.Vpc('vpc-d91a23bf')

In [192]:
vvv.id

'vpc-d91a23bf'

In [ ]:
ec2.SecurityGroup

In [186]:
ec2.create_placement_group(GroupName='hi',Strategy='cluster')

ClientError: An error occurred (InvalidPlacementGroup.Duplicate) when calling the CreatePlacementGroup operation: The placement group 'hi' already exists.

In [19]:
ig = ec2.create_internet_gateway()
vpc.attach_internet_gateway(InternetGatewayId=ig.id)
print(ig.id)

route_table = vpc.create_route_table()
route = route_table.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=ig.id
)
print(route_table.id)

igw-f976c69d
rtb-eea42389


In [161]:
gateway = ec2.create_internet_gateway()

In [312]:
gateway.attach_to_vpc?

In [163]:
vpc.id

'vpc-14876473'

In [165]:
  subnet = vpc.create_subnet(CidrBlock='192.168.2.0/24')

In [166]:
subnet.state

'pending'

In [195]:
client.describe_key_pairs()

{'KeyPairs': [{'KeyFingerprint': '28:6e:51:d6:81:3d:51:22:4b:20:4a:fb:cf:9b:1f:9f:1d:2a:a9:94',
   'KeyName': 'aws_pach_api_service'},
  {'KeyFingerprint': '6b:07:eb:f9:ab:ff:4f:18:3c:65:6e:99:05:01:21:fc:71:be:be:13',
   'KeyName': 'aws_pach_api_service_stage'},
  {'KeyFingerprint': 'f7:9a:33:d7:72:63:fe:27:30:9a:3b:49:e5:fa:dd:0b:9a:31:77:7a',
   'KeyName': 'brendan'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
   'KeyName': 'kubernetes.0726aa4c-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
   'KeyName': 'kubernetes.282e747a-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
   'KeyName': 'kubernetes.6dbcbbe2-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:1

In [197]:
keypair=ec2.KeyPair('yaroslav')

In [199]:
client.describe_placement_groups()

{'PlacementGroups': [{'GroupName': 'beefy3',
   'State': 'available',
   'Strategy': 'cluster'},
  {'GroupName': 'beefy2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy_head', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'default', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'test3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon1', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded', 'State': 'available', 'Strategy': 'cluster'}],
 'ResponseMetadata': {'HTTPHead

In [205]:
plgroup=ec2.PlacementGroup('beefy2')

In [209]:
grp=ec2.PlacementGroup('amazon')

In [210]:
grp.load()

ClientError: An error occurred (InvalidPlacementGroup.Unknown) when calling the DescribePlacementGroups operation: The Placement Group 'amazon' is unknown.

In [214]:
ec2.create_placement_group(GroupName="somegroup",Strategy="cluster")

ec2.PlacementGroup(name='somegroup')

In [217]:
grp=ec2.PlacementGroup('somegroup')

In [218]:
grp.load()

In [222]:
grp.name

'somegroup'

In [216]:
client.describe_placement_groups()

{'PlacementGroups': [{'GroupName': 'beefy2',
   'State': 'available',
   'Strategy': 'cluster'},
  {'GroupName': 'sharded', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy_head', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon1', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'test3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'default', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon3', 'State': 'available', 'Strategy': 'cluster'}],
 'ResponseMetadata': {'HTTPHead

In [182]:
security_group.create_tags(Tags=[{"Key": "Name", "Value": "default_vpc"}])

[ec2.Tag(resource_id='sg-c58e97a3', key='Name', value='default_vpc')]

In [194]:
ec2.SecurityGroup('sg-c58e97a3')

ec2.SecurityGroup(id='sg-c58e97a3')

In [183]:
security_group.create_tags?

In [170]:
route_tables = list(vpc.route_tables.all())

In [172]:
route_tables[0].routes

[ec2.Route(route_table_id='rtb-45a42322', destination_cidr_block='192.168.0.0/16')]

In [20]:
# create subnet
subnet = ec2.create_subnet(CidrBlock='192.168.1.0/24', VpcId=vpc.id)
print(subnet.id)

subnet-0d1f9c6a


In [21]:
# associate the route table with the subnet
route_table.associate_with_subnet(SubnetId=subnet.id)

ec2.RouteTableAssociation(id='rtbassoc-680ae60e')

In [29]:
# Create sec group
sec_group = ec2.create_security_group(
    GroupName='nexus', Description='nexus group', VpcId=vpc.id)

In [30]:
# enable pinging
sec_group.authorize_ingress(
    CidrIp='0.0.0.0/0',
    IpProtocol='icmp',
    FromPort=-1,
    ToPort=-1
)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 02:56:09 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '9be7631e-7150-4029-98d3-5aac5e6d36b8',
  'RetryAttempts': 0}}

In [ ]:
de

In [106]:
client.describe_security_groups()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 05:57:22 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '73a07462-2f68-4655-b0d6-a41eea1c4db1',
  'RetryAttempts': 0},
 'SecurityGroups': [{'Description': 'slice_0 sec group',
   'GroupId': 'sg-39f9e55f',
   'GroupName': 'slice_0',
   'IpPermissions': [{'FromPort': 8888,
     'IpProtocol': 'tcp',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 8888,
     'UserIdGroupPairs': []},
    {'FromPort': 22,
     'IpProtocol': 'tcp',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 22,
     'UserIdGroupPairs': []},
    {'FromPort': -1,
     'IpProtocol': 'icmp',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': -1,
     'Use

In [31]:
# enable SSH
sec_group.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=22,ToPort=22)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 02:56:10 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '0ce3a5f7-7922-4f59-a1f6-0a61f5924b40',
  'RetryAttempts': 0}}

In [32]:
# ipython notebook and tensorboard ports
response = sec_group.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=8888,ToPort=8888)

In [33]:
response['ResponseMetadata']['HTTPStatusCode']

200

In [103]:
sec_group.group_id

'sg-4af9e52c'

In [177]:
  security_group = ec2.create_security_group(
    GroupName="hi", Description="hi",
    VpcId=vpc.id)


In [179]:
    response = security_group.authorize_ingress(IpProtocol="tcp",
                                                CidrIp="0.0.0.0/0",
                                                FromPort=8888,ToPort=8888)


In [180]:
response

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 13 Dec 2017 21:20:45 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '14897fbc-151a-4c7f-b6a0-3f6d6edc0294',
  'RetryAttempts': 0}}

In [181]:
security_group.authorize_ingress?

In [224]:
grp.group_name

'somegroup'

In [229]:
from pprint import pprint as pp

In [233]:
[a['GroupName'] for a in client.describe_placement_groups()['PlacementGroups']]

['amazon',
 'sharded',
 'inet3',
 'amazon1',
 'amazon2',
 'sharded2',
 'default',
 'beefy_head',
 'beefy3',
 'amazon3',
 'test3',
 'inet',
 'beefy2']

NameError: name 'name' is not defined

In [253]:
client.describe_placement_groups()

{'PlacementGroups': [{'GroupName': 'default',
   'State': 'available',
   'Strategy': 'cluster'},
  {'GroupName': 'test3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon1', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy_head', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'beefy3', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'sharded2', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'inet', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'amazon', 'State': 'available', 'Strategy': 'cluster'}],
 'ResponseMetadata': {'HTTPHead

In [ ]:
ec2.

In [251]:
group = ec2.create_placement_group(GroupName='default', Strategy='cluster')

In [252]:
group.strategy

'cluster'

In [245]:
group = ec2.PlacementGroup('test_group')

In [246]:
group.strategy

'cluster'

In [313]:
ec2.create_instances?

In [424]:
grp = ec2.create_placement_group(GroupName="somegroup",Strategy="cluster")

ami="ami-45ead225"
instances = ec2.create_instances(
    ImageId=ami, InstanceType='c5.large', MaxCount=1, MinCount=1,
    NetworkInterfaces=[{'SubnetId': subnet.id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [sec_group.group_id]}],
    Placement={'GroupName': grp.group_name})

NameError: name 'sec_group' is not defined

In [173]:
client.describe_security_groups()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 13 Dec 2017 21:16:43 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '8be31302-d613-4b55-b0b5-33c984f2c84f',
  'RetryAttempts': 0},
 'SecurityGroups': [{'Description': 'default VPC security group',
   'GroupId': 'sg-019aea7c',
   'GroupName': 'default',
   'IpPermissions': [{'IpProtocol': '-1',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': [{'GroupId': 'sg-019aea7c',
       'UserId': '316880547378'}]}],
   'IpPermissionsEgress': [{'IpProtocol': '-1',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': []}],
   'OwnerId': '316880547378',
   'VpcId': 'vpc-14500972'},
  {'Description': 'CVC-baseline.  Allows SSH, tensorboard, and jupyter access',
   'GroupId': 'sg-05aeb178',
   'GroupName': '

In [147]:
instances[0]

ec2.Instance(id='i-07adfb26e0fd845a9')

In [150]:
ii.load()

In [151]:
ii.public_ip_address

'13.56.149.107'

In [146]:
ii.wait_until_running()

In [140]:
list(instances)

[ec2.Instance(id='i-07adfb26e0fd845a9')]

In [47]:
import os
# http://boto3.readthedocs.io/en/latest/guide/ec2-example-key-pairs.html
KEY_PAIR_LOCATION=os.environ["HOME"]+'/.nexus.pem'
KEY_PAIR_NAME='nexus'

In [37]:
keypair = ec2.create_key_pair(KeyName='nexus')

In [ ]:
boto3.client('ec2', region_name=REGION)

In [41]:
open(KEY_PAIR_LOCATION, 'w').write(keypair.key_material)

1670

In [51]:
KEY_PAIR_LOCATION

'/Users/yaroslav/.nexus.pem'

In [110]:
client.describe_placement_groups()

{'PlacementGroups': [{'GroupName': 'test',
   'State': 'available',
   'Strategy': 'cluster'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 14:39:41 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '38ed4769-7f17-4a46-ba69-0c25a623959c',
  'RetryAttempts': 0}}

In [116]:
from collections import OrderedDict
def get_placement_group_dict():
  """Returns dictionary of {placement_group_name: state}"""

  client = create_ec2_client()
  response = client.describe_placement_groups()
  assert _is_good_response(response)

  result = OrderedDict()
  for placement_group_response in response['PlacementGroups']:
    key = placement_group_response['GroupName']
    assert key not in result, ("Duplicate placement group " +
                               placement_group_name)
    result[key] = placement_group_response['State']
  return result


In [117]:
get_placement_group_dict()

OrderedDict([('amazon2', 'available'),
             ('inet3', 'available'),
             ('default', 'available'),
             ('amazon3', 'available'),
             ('beefy', 'available'),
             ('inet', 'available'),
             ('amazon1', 'available'),
             ('beefy3', 'available'),
             ('beefy2', 'available')])

In [108]:
client.create_placement_group(GroupName="test",Strategy="cluster")

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 14:25:45 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'd28f424e-d4da-4227-a25b-0d45155136b8',
  'RetryAttempts': 0}}

In [ ]:
def _maybe_create_placement_group(name):
  client = boto3.client('ec2')
  try:
    client.describe_placement_groups(GroupNames=[name])
  except boto3.exceptions.botocore.exceptions.ClientError as e:
    print("Creating placement group: "+name)
    res = client.create_placement_group(GroupName=name, Strategy='cluster')

  counter = 0
  while True:
    try:
      res = client.describe_placement_groups(GroupNames=[name])
      if res['PlacementGroups'][0]['State'] == 'available':
        print("Found placement group: "+name)
        break
    except Exception as e:
      print(e)
    counter = counter + 1
    if counter >= 10:
      print('Failed to create placement group %s' % name)
    time.sleep(TIMEOUT_SEC)


In [52]:
os.system('chmod 400 %s'%(KEY_PAIR_LOCATION,))

0

In [ ]:
# todo: assert result is 0

In [99]:
def _is_good_response(response):
  return response["ResponseMetadata"]['HTTPStatusCode'] == 200


In [100]:
def get_keypair_dict():
  """Returns dictionry of {keypairname: fingerprint}."""
  
  client = create_ec2_client()
  response=client.describe_key_pairs()
  assert _is_good_response(response)
  
  from collections import defaultdict
  result = {}
  for keypair in response['KeyPairs']:
    keypair_name = keypair.get('KeyName', '')
    assert keypair_name not in result, "Duplicate key "+keypair_name
    result[keypair_name] = keypair.get('KeyFingerprint','')
  return result


In [101]:
get_keypair_dict()

{'aws_pach_api_service': '28:6e:51:d6:81:3d:51:22:4b:20:4a:fb:cf:9b:1f:9f:1d:2a:a9:94',
 'brendan': 'f7:9a:33:d7:72:63:fe:27:30:9a:3b:49:e5:fa:dd:0b:9a:31:77:7a',
 'kubernetes.0726aa4c-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
 'kubernetes.282e747a-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
 'kubernetes.6dbcbbe2-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
 'kubernetes.db5d0d76-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
 'nexus-oregon': '42:79:9a:be:17:03:c9:de:dd:05:92:de:85:d7:29:07:cd:b7:a0:5b',
 'pach_service_stage': '4e:2b:72:59:5f:11:94:2d:5a:92:ba:10:2e:f0:10:da:22:03:d3:30',
 'yaroslav': '43:fe:a7:51:d7:ad:26:7c:ce:0e:94:53:3

In [96]:
client=boto3.client('ec2', region_name='us-west-1')
response=client.describe_key_pairs()
response

{'KeyPairs': [{'KeyFingerprint': 'e4:fc:cf:7f:78:f2:ce:0d:5e:c9:e3:12:85:72:68:b1:ad:47:38:55',
   'KeyName': 'nexus'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 12 Dec 2017 05:42:01 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'a765e248-3fe2-4d41-b854-2d5eba7a2114',
  'RetryAttempts': 0}}

In [46]:
for r in response['KeyPairs']:
  print(r['KeyName'])

nexus


In [102]:
vpc

ec2.SecurityGroup(id='sg-4af9e52c')

In [265]:
subnet

ec2.Subnet(id='subnet-bca924db')

In [274]:
vpc.get_available_subresources()

[]

In [278]:
subnets=list(vpc.subnets.all())

In [279]:
subnets[0]

ec2.Subnet(id='subnet-bca924db')

In [268]:
vpc.tags

[{'Key': 'Name', 'Value': 'nexus_vpc'}]

In [48]:
# create again with security key
ami="ami-45ead225"
instances = ec2.create_instances(
    ImageId=ami, InstanceType='t2.micro', MaxCount=1, MinCount=1,KeyName=KEY_PAIR_NAME,
    NetworkInterfaces=[{'SubnetId': subnet.id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [sec_group.group_id]}])

In [49]:
# create again with security key
ami="ami-45ead225"
instances = ec2.create_instances(
    ImageId=ami, InstanceType='t2.micro', MaxCount=1, MinCount=1,KeyName=KEY_PAIR_NAME,
    NetworkInterfaces=[{'SubnetId': subnet.id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [sec_group.group_id]}])

## List/delete vpc's

In [305]:
def _create_ec2_client():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.client('ec2', region_name=REGION)


def _create_ec2_resource():
  REGION = os.environ['AWS_DEFAULT_REGION']
  return boto3.resource('ec2',region_name=REGION)

def get_vpc_dict():
  """Returns dictionary of named VPCs {name: vpc}

  Assert fails if there's more than one VPC with same name."""

  client = _create_ec2_client()
  response = client.describe_vpcs()
  assert _is_good_response(response)

  result = OrderedDict()
  ec2 = _create_ec2_resource()
  for vpc_response in response['Vpcs']:
    key = _get_name(vpc_response.get('Tags', []))
    if not key:  # skip VPC's that don't have a name assigned
      continue
      
    assert key not in result, ("Duplicate VPC group " + key)
    result[key] = ec2.Vpc(vpc_response['VpcId'])

  return result

def _get_name(tags):
  """Helper utility to extract name out of tags dictionary.
      [{'Key': 'Name', 'Value': 'nexus'}] -> 'nexus'
 
     Assert fails if there's more than one name.
     Returns '' if there's less than one name.
  """
  
  names = [entry['Value'] for entry in tags if entry['Key']=='Name']
  if not names:
    names = ['']
  assert len(names)==1, "have more than one name: "+str(names)
  return names[0]



In [290]:
client.describe_availability_zones()

{'AvailabilityZones': [{'Messages': [],
   'RegionName': 'us-west-1',
   'State': 'available',
   'ZoneName': 'us-west-1a'},
  {'Messages': [],
   'RegionName': 'us-west-1',
   'State': 'available',
   'ZoneName': 'us-west-1c'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 19 Dec 2017 17:22:45 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '329b7ce0-1020-48f3-9751-516408b14097',
  'RetryAttempts': 0}}

In [310]:
vpc=d['nexus']

In [311]:
vpc.delete()  # not working, delete through console instead https://console.aws.amazon.com/vpc/home?region=us-east-1#vpcs:

ClientError: An error occurred (DependencyViolation) when calling the DeleteVpc operation: The vpc 'vpc-febcfd86' has dependencies and cannot be deleted.

## Delete some vpc stuff

In [216]:
os.environ['AWS_DEFAULT_REGION']='us-east-1'

In [219]:
import sys
sys.path.insert(0, os.environ["HOME"]+'/git0/cluster')
import common_resources as c

In [233]:
  existing_vpcs = c.get_vpc_dict()

In [234]:
existing_vpcs

OrderedDict([('nexus', ec2.Vpc(id='vpc-1a054562')),
             ('nexus01', ec2.Vpc(id='vpc-c6c388be')),
             ('nexus2', ec2.Vpc(id='vpc-cbffbeb3')),
             ('nexus00', ec2.Vpc(id='vpc-58f3b820'))])

In [236]:
vpc=existing_vpcs['nexus00']

In [237]:
vpc.load()
route_tables = list(vpc.route_tables.all())

In [238]:
route_tables[0].delete()

ClientError: An error occurred (DependencyViolation) when calling the DeleteRouteTable operation: The routeTable 'rtb-8b57acf6' has dependencies and cannot be deleted.

In [240]:
route_tables[0].routes.

[ec2.Route(route_table_id='rtb-8b57acf6', destination_cidr_block='192.168.0.0/16')]

In [239]:
route_tables[1].delete()

ClientError: An error occurred (DependencyViolation) when calling the DeleteRouteTable operation: The routeTable 'rtb-9043b8ed' has dependencies and cannot be deleted.

## Route tables

In [267]:
route_tables = list(d['nexus'].route_tables.all())

In [269]:
route_table = route_tables[0]

In [271]:
route_table.id

'rtb-aeaa5cd3'

In [272]:
route_table.tags

[]

In [278]:
route_table.meta

ResourceMeta('ec2', identifiers=['id'])

## Internet gateways

In [274]:
internet_gateways = list(vpc.internet_gateways.all())

In [276]:
iga = internet_gateways[0]

In [277]:
iga.create_tags

## Keypairs

In [3]:
client.describe_key_pairs()

{'KeyPairs': [{'KeyFingerprint': '28:6e:51:d6:81:3d:51:22:4b:20:4a:fb:cf:9b:1f:9f:1d:2a:a9:94',
   'KeyName': 'aws_pach_api_service'},
  {'KeyFingerprint': '6b:07:eb:f9:ab:ff:4f:18:3c:65:6e:99:05:01:21:fc:71:be:be:13',
   'KeyName': 'aws_pach_api_service_stage'},
  {'KeyFingerprint': 'f7:9a:33:d7:72:63:fe:27:30:9a:3b:49:e5:fa:dd:0b:9a:31:77:7a',
   'KeyName': 'brendan'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
   'KeyName': 'kubernetes.0726aa4c-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
   'KeyName': 'kubernetes.282e747a-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:19:8d:1c:6d:4a',
   'KeyName': 'kubernetes.6dbcbbe2-pachydermcluster.kubernetes.com-79:6b:55:2e:89:4d:be:42:29:3b:ba:cd:22:c8:ee:58'},
  {'KeyFingerprint': '1d:61:59:c6:eb:c3:ca:c4:a9:7a:7a:1

In [2]:
keypairs = u.get_keypair_dict()

In [4]:
keypair = keypairs['nexus']

In [5]:
keypair.name

'nexus'

In [6]:
keypair.key_fingerprint

'2e:b3:5b:59:62:a4:b2:21:51:71:e3:88:3c:27:33:bb:3c:76:ea:63'

In [9]:
import socket
#socket.gethostname()
socket.getfqdn()

'mac.local'

In [16]:
import hashlib
hash_object = hashlib.md5(socket.gethostname().encode('ascii'))
hash_object.hexdigest()

'b0207aa6d182a61fb2a886607b046e11'

'b0207aa6d182a61fb2a886607b046e11'

In [14]:
import getpass
username = getpass.getuser()

In [15]:
username

'yaroslav'

## Enable DNS on VPC

In [437]:
vpc = u.get_vpc_dict()['nexus']

In [440]:
response = vpc.modify_attribute(EnableDnsHostnames={"Value":True})
response = vpc.modify_attribute(EnableDnsSupport={"Value":True})

In [441]:
response

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 20 Dec 2017 01:37:16 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'e52ad0de-54d2-4962-87d1-d59384c0bbf9',
  'RetryAttempts': 0}}

In [ ]:
ec2.Vpc('vpc-aaaaaaaa').describe_attribute(Attribute='enableDnsHostnames')['EnableDnsHostnames']['Value']


# Efs troubleshooting

In [442]:
os.environ['AWS_DEFAULT_REGION']

'us-east-1'

In [443]:
vpc = u.get_vpc_dict()['nexus']

In [446]:
vpc.describe_attribute(Attribute='enableDnsSupport')

{'EnableDnsSupport': {'Value': True},
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 20 Dec 2017 01:44:55 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '5e54000b-525f-42ce-a27f-ccfc34b1d4b3',
  'RetryAttempts': 0},
 'VpcId': 'vpc-26e6b35e'}

In [447]:
vpc.describe_attribute(Attribute='enableDnsHostnames')

{'EnableDnsHostnames': {'Value': True},
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 20 Dec 2017 01:45:12 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'c9fbb240-0ad9-4072-970d-ea2643176410',
  'RetryAttempts': 0},
 'VpcId': 'vpc-26e6b35e'}

In [448]:
efs_id = u.get_efs_dict()['nexus']

In [449]:
ec2 = u.create_ec2_resource()

In [450]:
instances = list(ec2.instances.all())

In [451]:
instance = instances[0]

In [482]:
zone = instance.placement['AvailabilityZone']

In [480]:
importlib.reload(u)
u.get_mount_targets_dict(efs_id)

OrderedDict([('us-east-1c', 'fsmt-c5454a8c'),
             ('us-east-1b', 'fsmt-c7454a8e'),
             ('us-east-1f', 'fsmt-d8454a91'),
             ('us-east-1a', 'fsmt-d9454a90'),
             ('us-east-1e', 'fsmt-da454a93'),
             ('us-east-1d', 'fsmt-db454a92')])

In [472]:
instance.security_groups

[{'GroupId': 'sg-3d2a2148', 'GroupName': 'nexus'}]

In [484]:
mount_target = u.get_mount_targets_dict(efs_id)[zone]

In [474]:
efs_client = u.create_efs_client()

In [486]:
efs_client.describe_mount_target_security_groups(MountTargetId=mount_target)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '34',
   'content-type': 'application/json',
   'date': 'Wed, 20 Dec 2017 02:04:55 GMT',
   'x-amzn-requestid': '2c486ab4-e52a-11e7-9043-3787982a1618'},
  'HTTPStatusCode': 200,
  'RequestId': '2c486ab4-e52a-11e7-9043-3787982a1618',
  'RetryAttempts': 0},
 'SecurityGroups': ['sg-3d2a2148']}

In [488]:
security_group = u.get_security_group_dict()['nexus']

In [489]:
security_group.ip_permissions

[{'FromPort': 8890,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 8890,
  'UserIdGroupPairs': []},
 {'FromPort': 8888,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 8888,
  'UserIdGroupPairs': []},
 {'FromPort': 6006,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 6006,
  'UserIdGroupPairs': []},
 {'FromPort': 22,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 22,
  'UserIdGroupPairs': []},
 {'FromPort': 6379,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 6379,
  'UserIdGroupPairs': []},
 {'FromPort': 6008,
  'IpProtocol': 'tcp',
  'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
  'Ipv6Ranges': [],
  'PrefixListIds': [],
  'ToPort': 6008,

In [490]:
security_group.authorize_ingress()

--source-group sg-1a2b3c4d

In [494]:
security_group.group_name

'nexus'

In [496]:
security_group.authorize_ingress?

In [497]:
security_group.authorize_ingress?

In [507]:
client.authorize_security_group_ingress(
                                        GroupId=security_group.id,
                                        IpProtocol='all', FromPort=0,
ToPort=0,SourceSecurityGroupName=security_group.group_id)

ClientError: An error occurred (InvalidGroup.NotFound) when calling the AuthorizeSecurityGroupIngress operation: The security group 'sg-3d2a2148' does not exist in default VPC 'vpc-79c4f61d'

In [514]:
2**16-1

65535

In [517]:
new_rule ={'FromPort': 0,
 'IpProtocol': 'tcp',
 'IpRanges': [],
 'PrefixListIds': [],
 'ToPort': 65535,
 'UserIdGroupPairs': [{'GroupId': security_group.id}]} #id in VPC2
security_group.authorize_ingress(IpPermissions=[new_rule]) 

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 20 Dec 2017 02:35:26 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '14836c3f-0ad1-4c14-9d93-332c642480e9',
  'RetryAttempts': 0}}

In [526]:
    for protocol in ['tcp', 'udp', 'icmp']:
      try:
        rule ={'FromPort': 0,
               'IpProtocol': protocol,
               'IpRanges': [],
               'PrefixListIds': [],
               'ToPort': 65535,
               'UserIdGroupPairs': [{'GroupId': security_group.id}]}
        security_group.authorize_ingress(IpPermissions=[new_rule])
      except Exception as e:
        if e.response['Error']['Code']=='InvalidPermission.Duplicate':
          print("Warning, got "+str(e))
        else:
          assert False, "Failed while authorizing ingress with "+str(e)
      


Warning, got An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: sg-3d2a2148, UDP, from port: 0, to port: 65535, ALLOW" already exists
Warning, got An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: sg-3d2a2148, UDP, from port: 0, to port: 65535, ALLOW" already exists
Warning, got An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: sg-3d2a2148, UDP, from port: 0, to port: 65535, ALLOW" already exists


In [527]:
u.get_efs_dict()

OrderedDict([('nexus', 'fs-fa5b62b3')])

In [536]:
  efs_id = u.get_efs_dict()["nexus"]
  region = 'us-east-1'
  dns = "{efs_id}.efs.{region}.amazonaws.com".format(**locals())


In [537]:
dns

'fs-fa5b62b3.efs.us-east-1.amazonaws.com'

In [534]:
"{a}".format(**locals())

'5'

In [530]:
locals()["efs_id"]

'fs-fa5b62b3'

In [525]:
a.response['Error']['Code']=='InvalidPermission.Duplicate'

True

In [ ]:
new_rule ={'FromPort': 0,
 'IpProtocol': 'udp',
 'IpRanges': [],
 'PrefixListIds': [],
 'ToPort': 65535,
 'UserIdGroupPairs': [{'GroupId': security_group.id}]} #id in VPC2
security_group.authorize_ingress(IpPermissions=[new_rule]) 

In [511]:
client.authorize_security_group_ingress?

In [505]:
security_group.id

'sg-3d2a2148'

In [ ]:
aws ec2 authorize-security-group-ingress --group-id sg-3d2a2148 --protocol tcp --port 80 --source-group sg-3d2a2148

In [501]:
client.authorize_security_group_ingress()

In [498]:
    security_group.authorize_ingress(
      SourceSecurityGroupName=security_group.id,
      IpProtocol='All',
      FromPort=-1,
      ToPort=-1
    )


ClientError: An error occurred (VPCIdNotSpecified) when calling the AuthorizeSecurityGroupIngress operation: No default VPC for this user

In [500]:
security_group.vpc_id

'vpc-26e6b35e'

In [ ]:
  ec2 = boto3.resource('ec2')

  # TODO: add waiting so that instances that "initializing" are supported
  instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

  result = []
  for i in instances.all():
    inst_name = u.get_name(i.tags)
    key_name = i.key_name

    if inst_name == name:
      if key_name != KEY_NAME:
        print("name matches, but key name %s doesn't match %s, skipping"%(key_name, KEY_NAME))
        continue
      result.append(i)

    ec2 = boto3.resource('ec2')
  instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

  result = []
  for i in instances:
    names = []
    if i.tags:
      names = [tag['Value'] for tag in i.tags if tag['Key'] == 'Name']
    key_name = i.key_name

    assert len(names) <= 1
    if names:
      inst_name = names[0]
    else:
      inst_name = ''
    if inst_name == name:
      if key_name != KEY_NAME:
        print("name matches, but key name %s doesn't match %s, skipping"%(key_name, KEY_NAME))
        continue
      result.append(i)
  return result


## Create EFS

In [291]:
os.environ['AWS_DEFAULT_REGION']='us-east-1'
efs_client=_create_efs_client()

In [292]:
efs_client.describe_file_systems()

{'FileSystems': [],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '50',
   'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 17:23:01 GMT',
   'x-amzn-requestid': '439df84c-e4e1-11e7-bfec-e1d94534db9c'},
  'HTTPStatusCode': 200,
  'RequestId': '439df84c-e4e1-11e7-bfec-e1d94534db9c',
  'RetryAttempts': 0}}

In [293]:
c.get_security_group_dict()

OrderedDict([('nexus2', ec2.SecurityGroup(id='sg-50cfe125')),
             ('nexus00', ec2.SecurityGroup(id='sg-891a13fc')),
             ('nexus', ec2.SecurityGroup(id='sg-d0cae4a5'))])

In [340]:
token = str(int(time.time()*1e6)) # epoch usec

In [346]:
a = None
try:
  response = efs_client.create_file_system(CreationToken=token, PerformanceMode='generalPurpose')
except Exception as e:
  print(e)
  a = e

An error occurred (FileSystemAlreadyExists) when calling the CreateFileSystem operation: None


In [351]:
a.response['Error']['Code']

'FileSystemAlreadyExists'

In [366]:
importlib.reload(u)

<module 'util' from '/Users/yaroslav/git0/cluster/util.py'>

In [367]:
u.create_efs('nexus01')

In [368]:
u.get_efs_dict()

OrderedDict([('nexus01', 'fs-0f95ab46'), ('nexus00', 'fs-8397a9ca')])

In [352]:
efs.

In [32]:
response = efs_client.create_tags(FileSystemId=fs_id, Tags=[{"Key":"name", "Value":"test_efs"}])

In [34]:
fs_id

'fs-4bf940e2'

In [363]:
fs_id = u.get1(efs_client.describe_file_systems()['FileSystems'], FileSystemId=-1, CreationToken=token)

In [364]:
efs_client.create_tags(FileSystemId=fs_id, Tags=u.make_name('nexus00'))

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 18:02:12 GMT',
   'x-amzn-requestid': 'bd865df2-e4e6-11e7-9a73-2b548ec68a6f'},
  'HTTPStatusCode': 204,
  'RequestId': 'bd865df2-e4e6-11e7-9a73-2b548ec68a6f',
  'RetryAttempts': 0}}

In [365]:
u.get_efs_dict()

OrderedDict([('nexus00', 'fs-8397a9ca')])

In [307]:
fs_id = c.get1(efs_client.describe_file_systems()['FileSystems'], FileSystemId=-1, CreationToken=token)

In [310]:
efs_client.create_tags(FileSystemId=fs_id, Tags=c.make_name('nexus00'))

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 17:29:19 GMT',
   'x-amzn-requestid': '25dd20d1-e4e2-11e7-851c-5bda6b65b79c'},
  'HTTPStatusCode': 204,
  'RequestId': '25dd20d1-e4e2-11e7-851c-5bda6b65b79c',
  'RetryAttempts': 0}}

In [313]:
response = efs_client.describe_file_systems()
for efs_response in response['FileSystems']:
  print(efs_response['FileSystemId'])
  print(efs_client.describe_tags(FileSystemId=efs_response['FileSystemId']))

fs-4498a60d
{'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '46bef164-e4e2-11e7-978b-8d2563c5a66a', 'HTTPHeaders': {'x-amzn-requestid': '46bef164-e4e2-11e7-978b-8d2563c5a66a', 'content-type': 'application/json', 'date': 'Tue, 19 Dec 2017 17:30:15 GMT', 'content-length': '75'}}, 'Tags': [{'Value': 'nexus00', 'Key': 'Name'}]}


In [326]:
import util as u

In [334]:
  """Returns dictionary of {efs_name: efs_id}"""
  # there's no EC2 resource for EFS objects, so return EFS_ID instead
  # https://stackoverflow.com/questions/47870342/no-ec2-resource-for-efs-objects

  efs_client = u.create_efs_client()
  response = efs_client.describe_file_systems()
  assert u.is_good_response(response)
  result = OrderedDict()
  for efs_response in response['FileSystems']:
    fs_id = efs_response['FileSystemId']
    tag_response = efs_client.describe_tags(FileSystemId=fs_id)
    assert u.is_good_response(tag_response)
    key = u.get_name(tag_response['Tags'])
    if not key:   # skip EFS's without a name
      continue
    assert key not in result
    result[key] = fs_id


In [370]:
fs_id

'fs-8397a9ca'

In [371]:
efs_client.create_mount_target?

In [373]:
list(vpc.subnets.all())

[ec2.Subnet(id='subnet-f5e248a8')]

In [374]:
for subnet in vpc.subnets.all():
  print(subnet.id)

subnet-f5e248a8


In [388]:
efs_client.describe_tags(FileSystemId='fs-8397a9ca')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '75',
   'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 18:17:51 GMT',
   'x-amzn-requestid': 'ece8b9f3-e4e8-11e7-8125-435d90b564e8'},
  'HTTPStatusCode': 200,
  'RequestId': 'ece8b9f3-e4e8-11e7-8125-435d90b564e8',
  'RetryAttempts': 0},
 'Tags': [{'Key': 'Name', 'Value': 'nexus00'}]}

In [386]:
efs_client.describe_file_systems()

{'FileSystems': [{'CreationTime': datetime.datetime(2017, 12, 19, 10, 3, 44, tzinfo=tzlocal()),
   'CreationToken': '1513706624330134',
   'Encrypted': False,
   'FileSystemId': 'fs-0f95ab46',
   'LifeCycleState': 'available',
   'Name': 'nexus01',
   'NumberOfMountTargets': 0,
   'OwnerId': '316880547378',
   'PerformanceMode': 'generalPurpose',
   'SizeInBytes': {'Value': 6144}},
  {'CreationTime': datetime.datetime(2017, 12, 19, 9, 52, 33, tzinfo=tzlocal()),
   'CreationToken': '1513705952536349',
   'Encrypted': False,
   'FileSystemId': 'fs-8397a9ca',
   'LifeCycleState': 'available',
   'Name': 'nexus00',
   'NumberOfMountTargets': 1,
   'OwnerId': '316880547378',
   'PerformanceMode': 'generalPurpose',
   'SizeInBytes': {'Value': 6144}},
  {'CreationTime': datetime.datetime(2017, 12, 19, 9, 52, 18, tzinfo=tzlocal()),
   'CreationToken': '1513704300869863',
   'Encrypted': False,
   'FileSystemId': 'fs-8897a9c1',
   'LifeCycleState': 'available',
   'NumberOfMountTargets': 0,
   

In [ ]:
efs_client.create_mount_target(fs_id

In [379]:
security_group=u.get_security_group_dict()['nexus']
subnet = list(vpc.subnets.all())[0]

In [380]:
efs_client.create_mount_target(FileSystemId=fs_id, SubnetId=subnet.id, SecurityGroups=[security_group.id])

{'FileSystemId': 'fs-8397a9ca',
 'IpAddress': '192.168.1.217',
 'LifeCycleState': 'creating',
 'MountTargetId': 'fsmt-be9c90f7',
 'NetworkInterfaceId': 'eni-6083e1cd',
 'OwnerId': '316880547378',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '208',
   'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 18:10:36 GMT',
   'x-amzn-requestid': 'e96b9703-e4e7-11e7-8125-435d90b564e8'},
  'HTTPStatusCode': 200,
  'RequestId': 'e96b9703-e4e7-11e7-8125-435d90b564e8',
  'RetryAttempts': 0},
 'SubnetId': 'subnet-f5e248a8'}

In [393]:
efs_client.describe_mount_targets(FileSystemId=fs_id)

{'MountTargets': [{'FileSystemId': 'fs-8397a9ca',
   'IpAddress': '192.168.1.217',
   'LifeCycleState': 'available',
   'MountTargetId': 'fsmt-be9c90f7',
   'NetworkInterfaceId': 'eni-6083e1cd',
   'OwnerId': '316880547378',
   'SubnetId': 'subnet-f5e248a8'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '260',
   'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 18:23:30 GMT',
   'x-amzn-requestid': 'b765e2c5-e4e9-11e7-a0f8-71380bab6b2b'},
  'HTTPStatusCode': 200,
  'RequestId': 'b765e2c5-e4e9-11e7-a0f8-71380bab6b2b',
  'RetryAttempts': 0}}

In [406]:
efs_client.delete_file_system(FileSystemId='fs-8397a9ca')

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 19 Dec 2017 18:34:11 GMT',
   'x-amzn-requestid': '34e2dd88-e4eb-11e7-9723-2f715784a176'},
  'HTTPStatusCode': 204,
  'RequestId': '34e2dd88-e4eb-11e7-9723-2f715784a176',
  'RetryAttempts': 0}}

In [407]:
subnet.availability_zone

'us-east-1c'

In [405]:
efs_client.delete_mount_target(MountTargetId='fsmt-be9c90f7')

MountTargetNotFound: An error occurred (MountTargetNotFound) when calling the DeleteMountTarget operation: mount target does not exist

In [409]:
efs_client.

TypeError: get_waiter() missing 1 required positional argument: 'waiter_name'

In [396]:
ec2 = u.create_ec2_resource()

In [397]:
subnet = ec2.Subnet('subnet-f5e248a8')

In [398]:
subnet.availability_zone

'us-east-1c'

# Subnet troubleshooting

In [4]:
good_subnet_id='subnet-4862bf20'
bad_subnet_id='subnet-79920c56'

In [20]:
ec2_resource1=boto3.resource('ec2', 'us-east-1')
bad_subnet=ec2_resource1.Subnet('subnet-79920c56')
client1 = boto3.client('ec2', 'us-east-1')

ec2_resource2=boto3.resource('ec2', 'us-east-2')
good_subnet=ec2_resource2.Subnet('subnet-4862bf20')

In [22]:
bad_subnet.map_public_ip_on_launch

True

In [16]:
good_subnet.map_public_ip_on_launch

False

In [17]:
good_subnet.map_public_ip_on_launch=True

AttributeError: can't set attribute

In [21]:
response = client1.modify_subnet_attribute(
    MapPublicIpOnLaunch={
        'Value': True
    },
    SubnetId='subnet-79920c56'
)


In [ ]:
boto3.resource('ec2', 'us-east-1')

# Queryin instances

In [4]:
import os, sys, boto3
os.environ['AWS_DEFAULT_REGION']='us-west-2'
region = os.environ['AWS_DEFAULT_REGION']
client = boto3.client('ec2', region_name=region)
ec2 = u.create_ec2_resource()
response = client.describe_instances()

In [5]:
instances = list(ec2.instances.all())

In [6]:
instance=instances[0]

In [7]:
instance.public_ip_address

'34.216.9.68'

In [9]:
instance.instance_type

'r4.xlarge'

In [31]:
ec2.create_instances?

In [2]:
client.describe_instances()

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
       'Ebs': {'AttachTime': datetime.datetime(2017, 12, 26, 4, 19, 12, tzinfo=tzutc()),
        'DeleteOnTermination': True,
        'Status': 'attached',
        'VolumeId': 'vol-08604c56267074557'}},
      {'DeviceName': '/dev/xvdbo',
       'Ebs': {'AttachTime': datetime.datetime(2017, 12, 26, 4, 25, 16, tzinfo=tzutc()),
        'DeleteOnTermination': False,
        'Status': 'attached',
        'VolumeId': 'vol-031c9be6d29b8fac4'}}],
     'ClientToken': '21d58244-2ecc-0d59-b648-a3f5eb6950c9_subnet-117af859_1',
     'EbsOptimized': False,
     'Hypervisor': 'xen',
     'IamInstanceProfile': {'Arn': 'arn:aws:iam::316880547378:instance-profile/nodes.0726aa4c-pachydermcluster.kubernetes.com',
      'Id': 'AIPAJVWHPW5ZLSTJ2MT3Y'},
     'ImageId': 'ami-2fa0ba56',
     'InstanceId': 'i-056c288cf6677c393',
     'InstanceType': 'r4.

In [14]:
sys.path.insert(0, os.environ['HOME']+'/git0/cluster')
import util as u

In [15]:
u.get_name(instance.tags)

'sanc00'

In [18]:
instance.launch_time

datetime.datetime(2017, 12, 22, 0, 50, 10, tzinfo=tzutc())

In [ ]:
instance.

In [16]:
instance.launch_time

datetime.datetime(2017, 12, 22, 0, 50, 10, tzinfo=tzutc())

In [12]:
instance.tags

[{'Key': 'Name', 'Value': 'sanc00'}]

In [155]:
ami="ami-45ead225"
instances = ec2.create_instances(
    ImageId=ami, InstanceType='t2.micro', MaxCount=1, MinCount=1,
    NetworkInterfaces=[{'SubnetId': subnet.id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [sec_group.group_id]}])

In [538]:
instances[0].id

'i-00b80c5e0d4981f6b'

In [159]:
instance = instances[0]

In [540]:
instance.load()

In [542]:
instance.iam_instance_profile

AttributeError: 'NoneType' object has no attribute 'get'

# Uninitialize many instances

In [4]:
import os,boto3
os.environ['AWS_DEFAULT_REGION']='us-east-1'
region = os.environ['AWS_DEFAULT_REGION']
client = boto3.client('ec2', region_name=region)
ec2 = boto3.resource('ec2', region_name=region)

for instance in ec2.instances.all():
  if instance.state['Name'] == 'running':
    print(instance.id)


i-05a227278056dea5e
i-0d32740b5d1a6b0b8
i-0f2fc22c05c1e523d
i-02c967a42d0542711


In [6]:
instance=list(ec2.instances.all())[0]

In [7]:
instance.public_ip_address

'54.224.204.197'

In [8]:
instance.key_name

'nexus'

# Job level debugging

In [15]:
import sys
sys.path.insert(0, os.environ['HOME']+'/git0/cluster')
import aws

In [17]:
worker_job = aws.server_job('cifar-worker', num_tasks=1,availability_zone='us-east-1c')

Found existing job cifar-worker


In [20]:
worker_job.wait_until_ready()
worker_task = worker_job.tasks[0]

2017-12-27 04:50:31.728987 cifar-worker:0: (192.168.35.110) Running initialize
ssh_to_host ubuntu@54.83.131.12
2017-12-27 04:50:33.672796 cifar-worker:0: (192.168.35.110) Checking for initialized file
2017-12-27 04:50:34.239068 cifar-worker:0: (192.168.35.110) downloading /tmp/is_initialized to /tmp/tasklogs/0-1514346633855078.is_initialized
2017-12-27 04:50:35.107455 cifar-worker:0: (192.168.35.110) reusing previous initialized state
2017-12-27 04:50:35.274303 cifar-worker:0: (192.168.35.110) run_sync: tmux kill-session -t tmux
2017-12-27 04:50:35.941431 cifar-worker:0: (192.168.35.110) run_sync returned: 
2017-12-27 04:50:36.111033 cifar-worker:0: (192.168.35.110) run_sync: tmux new-session -s tmux -n 0 -d
2017-12-27 04:50:36.651738 cifar-worker:0: (192.168.35.110) run_sync returned: 
2017-12-27 04:50:37.012613 cifar-worker:0: (192.168.35.110) Checking for initialized file
2017-12-27 04:50:37.528438 cifar-worker:0: (192.168.35.110) downloading /tmp/is_initialized to /tmp/tasklogs/0-1

In [21]:
worker_task.run('ls')

2017-12-27 04:50:46.878479 cifar-worker:0: (192.168.35.110) tmux> ls
2017-12-27 04:50:48.154443 cifar-worker:0: (192.168.35.110) run_sync: mkdir -p /tmp/tmux
2017-12-27 04:50:48.933499 cifar-worker:0: (192.168.35.110) run_sync returned: 
2017-12-27 04:50:49.086868 cifar-worker:0: (192.168.35.110) run_sync: tmux send-keys -t tmux:0 'ls; echo $? > /tmp/tmux/2.1514346649086852.out' Enter
2017-12-27 04:50:49.597799 cifar-worker:0: (192.168.35.110) run_sync returned: 
2017-12-27 04:50:50.730751 cifar-worker:0: (192.168.35.110) downloading /tmp/tmux/2.1514346649086852.out to /tmp/tmux/1514346650231037


In [22]:
open('/tmp/hi.txt','w').write("hello")
worker_task.upload('/tmp/hi.txt', 'cifar10_main.py')

2017-12-27 04:51:44.540383 cifar-worker:0: (192.168.35.110) Remote file cifar10_main.py exists, skipping


# Time conversions

In [7]:
from operator import itemgetter

ec2 = u.create_ec2_resource()
instances = [(u.seconds_from_datetime(i.launch_time), i) for i in ec2.instances.all()]
sorted_instances = sorted(instances, key=itemgetter(0))


In [9]:
inst=sorted_instances[-1][1]

In [10]:
u.get_name(inst.tags)

'0.gpubox.deleteme'

In [11]:
inst.launch_time

datetime.datetime(2018, 1, 27, 0, 58, 46, tzinfo=tzutc())

In [12]:
import time

In [14]:
dt=inst.launch_time
time.mktime(dt.utctimetuple())

1517043526.0

In [20]:
from datetime import date

In [21]:
     import pytz
     utc = pytz.UTC
     utc.localize(date.fromtimestamp(1517043526.0))


AttributeError: 'datetime.date' object has no attribute 'tzinfo'

In [ ]:
inst.reboot()

# AMI retrieval

In [9]:
filter = {'Name': 'name', 'Values' : ['pytorch*']}
for i in ec2.images.filter(Filters = [filter], Owners=['self']):
  print(i.name, i.id)

pytorch.imagenet.source.v2 ami-78dfe807
pytorch.imagenet ami-a9bb9bd6


In [8]:
def lookup_ami(wildcard):
  filter = {'Name': 'name', 'Values' : [wildcard]}
  images = list(ec2.images.filter(Filters = [filter], Owners=['self']))
  assert len(images)<=1, "Multiple images match "+str(wildcard)
  assert len(images)>=0, "No images match "+str(wildcard)
  return i

'ami-a9bb9bd6'

In [ ]:
i

# Clean-up placement groups

In [10]:
client = u.create_ec2_client()

In [12]:
ec2 = u.create_ec2_resource()

In [ ]:
for group in ec2.placement_groups.all():
  try:
    print("Deleting "+group.name)
    group.delete()
  except:
    print("   Failed")
  else:
    print("   Succeeded")

Deleting tf
   Succeeded
Deleting run01
   Succeeded
Deleting c5
   Succeeded
Deleting psbench9
   Succeeded
Deleting gloop2p
   Succeeded
Deleting raybench44
   Succeeded
Deleting yaro16
   Succeeded
Deleting placement
   Succeeded
Deleting yaro_test
   Succeeded
Deleting tfbench
   Succeeded
Deleting psbench6
   Succeeded
Deleting psbench7
   Succeeded
Deleting yaro-friday-16
   Succeeded
Deleting simple
   Succeeded
Deleting tfbench44
   Succeeded
Deleting singlerun
   Failed
Deleting raybench20
   Succeeded
Deleting many02
   Succeeded
Deleting raybench2
   Succeeded
Deleting raybench17
   Succeeded
Deleting run03
   Succeeded
Deleting eight
   Succeeded
Deleting raybench3
   Succeeded
Deleting yaro-friday-8
   Succeeded
Deleting c5amazon00
   Succeeded
Deleting psbench13
   Succeeded
Deleting raybench4
   Succeeded
Deleting async
   Succeeded
Deleting psbench12
   Succeeded
Deleting nccl
   Succeeded
Deleting pytorch_cluster_f
   Succeeded
Deleting p2pgpu
   Succeeded
Deleting psb

In [16]:
group.name

'c5amazon00'

In [11]:
client.describe_placement_groups()

{'PlacementGroups': [{'GroupName': 'raybench3',
   'State': 'available',
   'Strategy': 'cluster'},
  {'GroupName': 'sixteen', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'singlerun', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'c5amazon00', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'tfbench', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'p2pgpu', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'tfbench4', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'psbench7', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'async', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'somegroup', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'psbench11', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'raybench16', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupName': 'raybench19', 'State': 'available', 'Strategy': 'cluster'},
  {'GroupN

In [ ]:
  client = u.create_ec2_client()
  try:
    # TODO: check that the error is actually about placement group
    client.describe_placement_groups(GroupNames=[name])
  except boto3.exceptions.botocore.exceptions.ClientError as e:
    # 
    print("Creating placement group: "+name)
    res = client.create_placement_group(GroupName=name, Strategy='cluster')


# Scratch

In [43]:
import tensorflow as tf

In [ ]:
tf.random

In [38]:
int(5) == 5

True

In [434]:
  name = 'gpubox00'
  KEY_NAME='nexus'
  ec2 = boto3.resource('ec2')
  instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

  result = []
  for i in instances.all():
    print(i)
    name = u.get_name(i.tags)
    key_name = i.key_name

    assert len(names) <= 1
    if names:
      inst_name = names[0]
    else:
      inst_name = ''
    if inst_name == name:
      if key_name != KEY_NAME:
        print("name matches, but key name %s doesn't match %s, skipping"%(key_name, KEY_NAME))
        continue
      result.append(i)

ec2.Instance(id='i-07a0cd1b69980ebc8')
ec2.Instance(id='i-0f99e16b39aacbca1')


In [436]:
ec2.create_vpc?

In [80]:
current_seconds=time.time();
remainder_micros=int(1e6*(current_seconds-int(current_seconds)))
time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_seconds))
full_time_str = "%s.%06d"%(time_str, remainder_micros)
full_time_str

'2017-12-04 17:31:27.013135'

'2017-12-04 17:31:14.381412'

In [37]:
os.listdir('.')

['plasma_manager91853411',
 'plasma_store10537880',
 'plasma_store18236878',
 'plasma_store88828496',
 'scheduler62714006',
 'plasma_store41416591',
 'plasma_manager69489913',
 'scheduler4710266',
 'scheduler8096252',
 'plasma_store25595318',
 'scheduler85908030',
 'scheduler37020026',
 'plasma_manager34279968',
 'plasma_manager16268969',
 'plasma_store61975526',
 'plasma_store59559392',
 'plasma_manager48870941',
 'plasma_manager4501529',
 'scheduler91987771',
 'plasma_store34948235',
 'com.apple.launchd.phjQmyOsHP',
 'scheduler85745644',
 'plasma_manager76483440',
 'plasma_manager98848736',
 'plasma_manager63116115',
 'at-hRSIKu',
 'scheduler35081158',
 'plasma_store13954523',
 'plasma_manager91698017',
 'plasma_manager95221948',
 'scheduler267754',
 'plasma_manager413673',
 'scheduler29933459',
 'scheduler69376320',
 'plasma_manager72378285',
 'scheduler46128327',
 'plasma_manager620644',
 'scheduler94136826',
 'raylogs',
 'scheduler10661819',
 'plasma_store4931042',
 'adobegc.log',

In [36]:
os.getcwd()

'/private/tmp'

In [35]:
os.chdir('/tmp')

In [32]:
KEY_NAME='yaroslav'

def lookup_aws_instances(name):
  """Returns all AWS instances for given job."""
  
  ec2 = boto3.resource('ec2')
  instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

  result = []
  for i in instances:
    names = []
    if i.tags:
      names = [tag['Value'] for tag in i.tags if tag['Key'] == 'Name']
    key_name = i.key_name

    assert len(names) <= 1
    if names:
      inst_name = names[0]
    else:
      inst_name = ''
    if inst_name == name:
      if key_name != KEY_NAME:
        print("name matches, but key name %s doesn't match %s, skipping"%(key_name, KEY_NAME))
        continue
      result.append(i)
  return result


In [34]:
instances=lookup_aws_instances("hello")

In [35]:
instance = instances[0]

In [37]:
instance.public_ip_address

'54.68.66.51'

In [38]:
import os

In [40]:
os.path.basename("https://gist.githubusercontent.com/robertnishihara/4e246d6942cd692a0838414ff01975c1/raw/486a6a3f8c5cfc3d198e73cea50507a829e0cc9d/async_sgd_benchmark.py")

'async_sgd_benchmark.py'

In [36]:
dir(instance)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_id',
 'ami_launch_index',
 'architecture',
 'attach_classic_link_vpc',
 'attach_volume',
 'block_device_mappings',
 'classic_address',
 'client_token',
 'console_output',
 'create_image',
 'create_tags',
 'delete_tags',
 'describe_attribute',
 'detach_classic_link_vpc',
 'detach_volume',
 'ebs_optimized',
 'elastic_gpu_associations',
 'ena_support',
 'get_available_subresources',
 'hypervisor',
 'iam_instance_profile',
 'id',
 'image',
 'image_id',
 'instance_id',
 'instance_lifecycle',
 'instance_type',
 'kernel_id',
 'key_name',
 'key_pair',
 'launch_time',
 'load',
 'meta',
 'modify_attribute',
 'monitor',
 'monitoring',
 'network_interfa

In [81]:
ec2 = boto3.resource('ec2')
aws_instances = ec2.create_instances?


In [ ]:
aws_instances = ec2.create_instances

In [ ]:
aws_instances = ec2.create_instances

In [43]:
ec2 = boto3.client('ec2')
response = ec2.describe_instances()

In [14]:
import boto3
ec2 = boto3.resource('ec2')
AMI = 'ami-9ddb0fe5'
KEY_NAME = 'yaroslav'  # AWS key-name to use
SECURITY_GROUP = 'open' # security group for all instances
placement_group='ebs-test'

ec2 = boto3.resource('ec2')
instances = ec2.create_instances(
ImageId=AMI,
InstanceType='c5.large',
MinCount=1,
MaxCount=1,
SecurityGroups=[SECURITY_GROUP],
KeyName=KEY_NAME)

In [27]:
    ec2 = boto3.resource('ec2')
    instances2 = ec2.create_instances(
      ImageId=AMI,
      InstanceType='c5.2xlarge',
      MinCount=4,
      MaxCount=4,
      SecurityGroups=[SECURITY_GROUP],
      KeyName=KEY_NAME)


In [25]:
instances

[ec2.Instance(id='i-037156ba385ad4fe5'),
 ec2.Instance(id='i-01c4a7e0f5ca56b32'),
 ec2.Instance(id='i-0805ec38fd3cf71b9'),
 ec2.Instance(id='i-09322ea431460f284'),
 ec2.Instance(id='i-00356bd9050161866'),
 ec2.Instance(id='i-0cdde35372adcf3d3'),
 ec2.Instance(id='i-0aae50b99f628f851'),
 ec2.Instance(id='i-0a2c0879c1d4118f6')]

In [20]:
[(i.id, tag['Value']) for i in instances for tag in i.tags if tag['Key'] == 'Name']

[('i-04043b4d7436be162', 'testjob'),
 ('i-04101fcf8a211e862', 'testjob'),
 ('i-0b1b3c658b51e3fe6',
  'master-us-west-2a.masters.90c444a2-pachydermcluster.kubernetes.com'),
 ('i-0aaa1f818181e50d0',
  'master-us-west-2a.masters.0726aa4c-pachydermcluster.kubernetes.com'),
 ('i-07f83369033a741a3', 'pachyderm-webservice'),
 ('i-08259f03e6c8061c3', 'testjob'),
 ('i-008877bc5b2a6ef43', 'testjob'),
 ('i-03baf8ed44431961c', 'nodes.0726aa4c-pachydermcluster.kubernetes.com'),
 ('i-0498d6623c44de7b7', 'nodes.0726aa4c-pachydermcluster.kubernetes.com'),
 ('i-02831a5e1defa3caa', 'nodes.90c444a2-pachydermcluster.kubernetes.com'),
 ('i-01c7202a3c06f35d3', 'nodes.90c444a2-pachydermcluster.kubernetes.com')]